In [1]:
!pip3 install networkx

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip3 install wiki_ru_wordnet

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip3 install conllu

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import os
from io import open
from conllu import parse_incr
from conllu import parse
import nltk
nltk.download('wordnet')
import re
import pickle
import networkx as nx
from nltk.corpus import wordnet as wn
from matplotlib import pyplot as plt
%matplotlib inline

from wiki_ru_wordnet import WikiWordnet
ruwn = WikiWordnet()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/victoriaregina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
with open('vocab.pickle', 'rb') as f:
    lemmas_vocab = pickle.load(f)

In [16]:
def UDPipe_processing(sentimentwords):
    
    food_vocab = {}
    service_vocab = {}
    massive = []
    
    for i, filename in enumerate(os.listdir('parsed texts')):
        
        food_or_service_d = {}

        data_file = open('parsed texts/' + filename, "r", encoding="utf-8")

        for id_sent, tokenlist in enumerate(parse_incr(data_file)):

            for token in tokenlist:

                if token['lemma'] in sentimentwords:

                    if token['upostag'] == 'VERB':

                          for dependent in tokenlist:

                                if dependent['upostag'] != 'PUNCT'and dependent['upostag'] != 'NUM' and dependent['head'] == token['id']:

                                    food_or_service_d[dependent['lemma'], id_sent] = token['lemma'] + '\t' + filename + '\t' + str(id_sent+1) + '\t' + str(token['id']) + '\t' + str(sentimentwords[token['lemma']])

                    else:

                        for head in tokenlist:

                            if token['head'] == head['id']:

                                food_or_service_d[head['lemma'], id_sent] = token['lemma'] + '\t' + filename + '\t'+  str(id_sent+1) + '\t' + str(token['id']) + '\t' + str(sentimentwords[token['lemma']])
        
        massiv = food_or_service(food_or_service_d, food_vocab, service_vocab)
        massive.append(massiv)
        file = open(filename.split('.')[0] + '.txt', 'w')
        for elem in massiv:
            file.write(elem)
            file.write('\n')
        file.close()
        
        file_food = open('Food_words.txt', 'w')
        for key in food_vocab:
            file_food.write('Food\t' + key + '\t' + food_vocab[key] + '\n')
        file_food.close()
        
        file_service = open('Service_words.txt', 'w')
        for k in service_vocab:
            file_service.write('Service\t' + k + '\t' + service_vocab[k] + '\n')
        file_service.close()
        
    return massive

In [17]:
def food_or_service(input_fs, food_vocab, service_vocab):
    
    a = []
    
    dict_food = ['еда','блюдо','кухня','салат','суп','пища']
    dict_service = ['обслуживание','официант','официант','персонал','работник','сотрудник']
    
    for key, value in input_fs.items():
        key = key[0]
        
        if key in dict_food:
            res = 'Food'
            make_tags_txt(value, res, a, food_vocab, service_vocab)
        
        elif key in dict_service:
            res = 'Service' 
            make_tags_txt(value, res, a, food_vocab, service_vocab)
        
        else:
            for i in range(1):
                try:
                    s = ruwn.get_synsets(key)[i]
                    for w in s.get_words():
                        for key_word in dict_food:
                              if key_word in w.definition():
                                res = 'Food'
                                make_tags_txt(value, res, a, food_vocab, service_vocab)
                    pass

                    for key_word in dict_service:
                          if key_word in w.definition():
                            res = 'Service'
                            make_tags_txt(value, res, a, food_vocab, service_vocab)
                
                except(IndexError):
                      pass
    return a

In [18]:
def make_tags_txt(text, res, array, food_vocab, service_vocab):
    
    meta_data = text.split('\t')
    
    lemma = meta_data[0]
    id_doc = meta_data[1].split('.')[0]
    id_sent = meta_data[2]
    id_token = meta_data[3]
    ton = meta_data[4].split('\n')[0]
    
    if res == 'Food':
        food_vocab[lemma] = ton
    if res == 'Service':
        service_vocab[lemma] = ton
    
    t = ''

    if len(array) > 0:
        previous = array[-1].split('\t')
        if str(previous[0]) == str(id_sent) and str(previous[2]) == str(res) and str(previous[3]) == str(ton):
            array.pop()
            if previous[1] != id_token:
                t = id_sent + '\t' + previous[1] + ',' + id_token + '\t' + res + '\t' + ton
                array.append(t)
            else:
                pass
        else:
            t = id_sent + '\t' + id_token + '\t' + res + '\t' + ton
            array.append(t)
    else:
        t = id_sent + '\t' + id_token + '\t' + res + '\t' + ton
        array.append(t)
    
    return array

In [19]:
m = UDPipe_processing(lemmas_vocab)